# main.ipynb
for TP2 IFT2015 2018

## Team members:
Qiang Ye(20139927) Xiao Ju(1037977)
<hr>

## Question 1 
### a) Implémenter la structure de données du graphe de Brujin à l’aide d’une table de hachage qui devra également être implantée. Le constructeur prendra en entrée un itérable sur des chaînes de caractères de longueur k.
Puisque l’itérable est de longueur arbitraire, vous devez redimensionner dynamiquement la table de hachage pour maintenir le facteur de charge sous 0.75.

**Answer a)**

Please read `graph.py` for details.


### b) Discuter des détails de votre implémentation en répondant aux questions suivantes
- Quelle fonction de hachage avez-vous utilisé? et dans quelle mesure est-elle spécifique au
problème?
- Quelle stratégie d’adressage avez-vous utilisé?

**Answer b)**

Please read `report` for details.

### c) Quel serait le principal gain d’utiliser une technique de hachage sensitif à la localité et de l’adressage linéaire pour ce problème?

**Answer c)**

Please read `report` for details.
<hr>

**Note: The following(next) cell is for running this pynb file on Google Colab platform. it doesn't need to be executed if this .ipynb is run on local computer.**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("drive/My Drive/Colab Notebooks/IFT2015/")
#check if you are in the right directory
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
GCF_000002985.6_WBcel235_rna.fna.gz  main.ipynb  __pycache__
graph.py			     out_dated	 reads.fastq.gz


<hr>
## Question 2

### a) Encoder tous les k-mers des fragments fournis dans le fichier FASTQ dans une instance du graphe De Brujin. Chaque fragment possède l − k + 1 k-mers.

**Answer a)**

**Step1**: Import `DeBrujinGraph` implemented in question1 and other necessary modules.

In [2]:
import gzip
from itertools import islice
from graph import DeBrujinGraph
import time
import sys
import random
import string

def show_live_info(info, desc_before = "", desc_after = ""):
    info_line = '\r' + desc_before +" " + str(info) + " " + desc_after + "\t"
    sys.stdout.flush()
    sys.stdout.write(info_line)

**Step2**: Use the method `read_fastq` provided to load sequences from file: 'reads.fastq.gz' and generate possible k-mers with k = 21.

In [8]:
def read_fastq(path):
    with gzip.open(path, 'rt') as f:
        for line in f:
            seqid, description = line[1:].rstrip().split(maxsplit = 1)
            sequence = f.readline().rstrip()
            _ = f.readline()
            quality = f.readline().rstrip()
            yield seqid, description, sequence, quality

In [9]:
k = 21
kmers = []

In [10]:
%%time
for _, _, seq, _ in read_fastq('reads.fastq.gz'):
    kmers.extend([seq[i:i+k] for i in range(len(seq) - k + 1)])

print(len(kmers), "k-mers")

8627600 k-mers
CPU times: user 1.71 s, sys: 136 ms, total: 1.84 s
Wall time: 1.84 s


**Step3**: Build an instance of `DeBrujinGraph` based on all k-mers.

In [6]:
%%time
length = len(kmers) 
#length = min(100000, len(kmers))
print(length)
# we can also remove the repeated kmers before passing them to graph
# kmers = list(set(kmers))

graph = DeBrujinGraph(nodes = kmers[:length], k = 21)

8627600
building nodes...
building edges...
CPU times: user 18min 19s, sys: 8.33 s, total: 18min 28s
Wall time: 18min 28s


**Step4**: View the node and edge counts of the graph generated.

In [7]:
print("node counts:", graph.node_count())
print("edge counts:", graph.edge_count())

node counts: 5126513
edge counts: 5091467


### b) Bonus: Implanter une table de hachage plus rapide que celle de CPython (i.e. celle utilisée par set et dict) pour stocker des k-mers pour les opérations effectuées en 2a.

**Answer b)**

Please read `report` for details.
<hr>

## Question 3 

### a)  Parcourir le graphe pour obtenir des segments contigüs que vous devrez stocker dans un fichier FASTA nommé contigs.fa avec un identifiant unique pour chacun. 
Commencer par identifier les noeuds qui n’ont pas de prédécesseurs pour débuter l’exploration.

Alternativement, vous pouvez explorer le graphe dans les deux directions en partant d’un point aléatoire sur chaque composante connexe.

**Answer a)**

**Step1**: Implement deep first search and breadth first search methods `DFS`, `DFS_complete`, `BFS`; these methods will be used to explore the connected components of the graph.

In [11]:
def DFS(g, u, discovered, dest_nodes):
    """deep first search of a graph.
    params
        g: a graph
        u: a node of graph g
        discovered: a dictionary marking where a edge is visited
        dest_nodes: a set of nodes who hae no successors in connected tree.
    return
        None, discovered and dest_nodes will be modified
    """
    for v in g.successors( u ):
        if v not in discovered:
            discovered[v] = u # u represents parent node of v
                              # v carries the edge information v[-1]
            dest_nodes.add(v)
            dest_nodes.discard(u)
            DFS( g, v, discovered, dest_nodes )
            
            
def BFS(g, s, discovered, dest_nodes):
    """deep first search of a graph.
    params
        g: a graph
        u: a node of graph g
        discovered: a dictionary marking where a edge is visited
        dest_nodes: a set of nodes who hae no successors in connected tree.
    return
        None, discovered and dest_nodes will be modified
    """
    level = [s]
    while len(level) > 0:
        next_level = []
        for u in level:
            for v in g.successors(u):
                if v not in discovered:
                    discovered[v] = u
                    dest_nodes.add(v)
                    dest_nodes.discard(u)
                    next_level.append(v)
        level = next_level  

**Step2**: Implement method `find_end_nodes` to find nodes without predecessors or without successors.

In [12]:
def find_end_nodes(g: DeBrujinGraph, is_start: bool = True):
    """find all nodes without predecessors or without successors controled by
    a bool parameter
    params
        g: graph
        is_start: bool, if True, find all nodes without predecessors; if False
            find all nodes without successors
    returns
        results, list of nodes
    """
    results = []
    func = g.predecessors
    if is_start == False:
        func = g.successors
    for node in g.nodes():
        if len(func(node)) == 0: # no predecessors
            results.append(node)
            show_live_info(len(results), "searching... ", "nodes found.")
    print()
    return results           

In [10]:
%%time
start_nodes = find_end_nodes(graph, is_start = True)

searching...  37319 nodes found.	
CPU times: user 3min 58s, sys: 14.1 s, total: 4min 12s
Wall time: 4min 39s


**Step3**: We can find all possible paths by starting from a node without predecessors and ended at a leaf node of a connected tree formed by a search method. Based on the paths found, a continuous sequence can be assembled.

In [13]:
def generate_sequence(g, u, v, discovered):
    """generate a contiguous sequence from node u to v based on the 
    discovered paths in graph
    params
        g: graph
        u: start(origin) node of a sequence, str
        v: end(destination) node of a sequence, str
        discovered: dictionary from a deep/breadth first search of a graph g
    returns
        segment: str, a string format of list of nodes. 
            example: "ACTGTCTATAT"
    """
    segment = []
    if u is None: # there is no start nodes
        return None   
    
    if v in discovered:
        walk = v
        while walk is not u:
            segment.append(walk[-1])
            walk = discovered[walk]
        # walk is now u
        
    segment.reverse()
    segment = u + "".join(segment)
    return segment


def generate_sequences(g, start_nodes):
    """generate all possible contiguous sequence from a directed graph with
    start nodes are given
    params
        g: graph
        start_nodes: list of start nodes, str
    returns
        segments: all possible contigous sequences, [str]
    """
    segments = []
    n_total_nodes = len(start_nodes)
    for i, start_node in enumerate(start_nodes):
        discovered = {start_node: '-'}
        dest_nodes = set()
        dest_nodes.add(start_node)
        # use DFS or BFS to construct a connected tree, and all leave nodes
        # circle path will be cut here.
        DFS(graph, start_node, discovered, dest_nodes)
        # BFS(graph, start_node, discovered, dest_nodes)
        for dest_node in dest_nodes:
            #print(start_node," -> ", dest_node)
            segment = generate_sequence(graph, start_node, dest_node, discovered)
            if segment is not None:
                segments.append(segment)
        
        show_live_info(len(segments), "generating sequence...", "generated.")
    print()
    return segments

**Step4**: Generate sequences assembled and write them to a file. We may need to change python's recursion limit.

In [12]:
%%time
# sys.getrecursionlimit()
sys.setrecursionlimit(10000)
segments = generate_sequences(graph, start_nodes)

generating sequence... 51468 generated.	
CPU times: user 4min 31s, sys: 14.4 s, total: 4min 45s
Wall time: 5min 10s


In [6]:
def get_identifier(k = 10):
    return ''.join(random.choices(string.ascii_uppercase, k = 10))

# We modified the original read_fasta method, as it only tackle the compressed
# and doesn't yield last sequence in the file.
def read_fasta(path):
    """read data from a fasta file. if the filename ends with 'gz', it will
    be considered a compressed file; otherwise, non compressed.
    
    params
        path: file path, str
    returns
        None
    """
    open_style = gzip.open          # zp file as default 
    if path[-2:] != 'gz':
        open_style = open           # if not file not ended with 'gz', use open
    with open_style(path, 'rt') as f:
        accession, description, seq = None, None, None
        for line in f:
            if line[0] == '>':
                # yield current record
                if accession is not None:
                    yield accession, description, seq
                    
                # start a new record
                accession, description = line[1:].rstrip().split(maxsplit=1)
                seq = ''
            else:
                seq += line.rstrip()
        yield accession, description, seq   # yield the last sequence
    return


def write_fasta(path, sequences):
    """write sequences data from a fasta file. if the filename ends with 'gz', 
    it will be considered a compressed file; otherwise, non compressed.
    params
        path: file path for writing, str
        sequences: list of sequence, [str]
    returns
        None
    """
    open_style = gzip.open
    if path[-2:] != 'gz': # not a compressed file
        open_style = open
    random.seed(123)
    desc_base = "Assembled by QiangYe and JuXiao 12/21/2018"
    identifiers = set()            # avoid repeatative identifiers
    max_bases_line = 80            # 80 bases maximal per line
    with open_style(path, "wt") as f:
        for sequence in sequences:
            identifier = get_identifier()
            while identifier in identifiers:  # assure each identifier is unique
                identifier = get_identifier()

            identifiers.add(identifier)
            desc = desc_base + " " + str(len(sequence)) + " bases"
            f.write(">" + identifier + " " + desc + "\n")
            echos = len(sequence) // max_bases_line
            if len(sequence) % max_bases_line > 0:
                echos += 1
            for echo in range(echos):
                start_index = echo * max_bases_line
                end_index = min(start_index + max_bases_line, len(sequence))
                f.write(sequence[start_index: end_index]+"\n")
    return

In [14]:
fasta_file = 'GCF_000002985.6_WBcel235_rna.fna.gz'
contigs_file = 'contigs_demo.fa' # change this file name

write_fasta(contigs_file, segments)
print(len(segments))
print(next(read_fasta(fasta_file)))
print(next(read_fasta(contigs_file)))

51468
('NM_001025782.2', 'Caenorhabditis elegans Phosphatidylinositol 3-kinase catalytic subunit type 3 (vps-34), partial mRNA', 'ATGCGAGTCTCCACGTCAGTCAACGGTGGTGTTGGGATTGTTTCGGCCTGTACCCGTTACTGTGTAGCTGATCCTGAACTTCTTCTAGAATCTCTGGCGGAAGTTAAACACAGTGCAATGACACGTCGAATCAGAGATGTTGAAGATGAGAGACATCGACAAGTAAAACCGAATAAACAAGCAAAGGATCGATTGGAGACTATTGTGAATCTTCCATCATCTCAAGTTCTTACAAGAGAACAAAGAGATTTAGTATGGAAATTCAGGCATTATTTAAGACAATTTCCAAAAGCTTTGAACAAATATCTACGATCAGTTAATTGGGTTCATCCACAAGAAGTAAAAACTGCATTAGCTCTGATGAATGATTGGGAGCTGATTGAAGCTGAAGATGCATTAGAATTACTTTCAAGTGCATTCACTCATCCGGCAGTTCGTGCATATTCTGTATCCCGTCTTCTAGAAGCTGCTTCTCCTGAGCAAGTCCTCCTATATCTTCCACAATTGGTTCAAGCTTTGAAATACGAACAAGGTCAACAACTGCCTGAAGAAGGAAATCCAGTGCCGGTGGTTTCTGAAGAAGAGGGAAAGATCCCATCAGTTGCAACTACGCCAACTGAAGAGCTAGAAGGACGAGATATGACAGTGGTTACGAAGAAAGAGGCTAGAAAGGCGGCTAGTGGAGATTTGGCCACCTTCCTGATTGACTATGCTCTTGCATCTCCAAAAGTATCCAACTATCTCTACTGGCATCTAAAAACTGAAATTGAATCAACAAAAGAAAGTAAAGAAGAACACTCGAAAATGTATCAAAATATCCAGGATAGATTGATGGAAGCACTTGTAAAGAGACCTGATACCCGTGCACAA

The shortest and largest lengths are :

In [15]:
lens = [len(segment) for segment in segments]
print(min(lens), max(lens))

24 5955


### b) Dans votre rapport, répondez aux questions suivantes:
- Quelle stratégie de parcours avez-vous adoptée?
- Comment les boucles sont-elles gérées durant le parcours?

**Answer b)**

Please read `report` for details.

<hr>

## Question 4 

### a) Comparer les séquences assemblées avec les séquences de références contenus dans le fichier FASTA. 

Vous pouvez utiliser string.find pour effectuer chaque paire de comparaison. Reporter chaque
occurence dans un fichier occurences.bed contenant les quatre (4) colonnes suivantes séparées
par des tabulations:
 - reference: identifiant du segment de référence
 - start: début de l’occurence basé sur 1 inclusif (une occurence à l’indice zéro devrait être en
position 1)
 - end: fin de l’occurence basé sur 1 inclusif
 - contig: identifiant du contig
 
**Answer a)** 
 
 **Step1**: implement methods to wirte matches to file and to read them.

In [1]:
def read_bed(path):
    with open(path) as f:
        for line in f:
            ref, start, end, name = line.rstrip().split('\t')
            yield ref, int(start) - 1, int(end), name
        
def write_bed(path, occurences):
    with open(path, "w") as f:
        for occur in occurences:
            line = ""
            for item in occur:
                line += str(item) + "\t"
            f.write(line+"\n")
    return

**Step2**: Compare sequences in assembeld fasta and references fasta.

First, read ferences sequences and assembled sequences from files:

In [16]:
%%time
fasta_file = 'GCF_000002985.6_WBcel235_rna.fna.gz'
contigs_file = 'contigs.fa'

refer_seqs = []
assem_seqs = []
for identifier, _, assem_seq in read_fasta(contigs_file):
    assem_seqs.append((identifier, assem_seq))
    
for identifier, _, ref_seq in read_fasta(fasta_file):
    refer_seqs.append((identifier, ref_seq))
    
print(len(refer_seqs), len(assem_seqs))

53144 51488
CPU times: user 948 ms, sys: 24.1 ms, total: 972 ms
Wall time: 968 ms


Then, compare sequences:

In [18]:
def compare_sequences(refer_seqs, assem_seqs):
    occurences, failures = [], []
    for identifier, assem_seq in assem_seqs:
        found = False
        for accession, seq in refer_seqs:
            index = seq.find(assem_seq)
            #index = sunday_find(seq, assem_seq)  # sunday_find
            if index > -1:
                found = True # this assembled seq has been found in references
                result_line = accession, index+1, index+len(assem_seq), identifier
                occurences.append(result_line)
                show_live_info(str(len(occurences)) + "/" + str(len(failures)),
                           desc_after = "  (matches/failures)")

        if not found: # assembled seq didn't match any sequence in fasta 
            failures.append((identifier, assem_seq))
            show_live_info(str(len(occurences)) + "/" + str(len(failures)),
                           desc_after = "  (matches/failures)")
    print()        
    return  occurences, failures

occurences, failures = [], []

In [26]:
%%time
occurences, failures = compare_sequences(refer_seqs, assem_seqs)

 72525/14025   (matches/failures)	CPU times: user 2h 17min 28s, sys: 55.7 s, total: 2h 18min 23s
Wall time: 2h 18min 27s


**Step3**: Wirte result to file.

In [0]:
sorted_occ = sorted(occurences)
# change filename to save current occurences in another file
write_bed("occurences_demo.bed", sorted_occ)

sorted_fail = sorted(failures)
# change filename to save current failures
write_bed("failure.bed", sorted_fail)

**Step4**: Check the information in `occurences.bed` file. 

Read two kind of sequences into dictionaries. Check the information in "occurences.bed" based on the sequences data.

In [4]:
def fasta_to_dict(fasta_file):
    """build a dict from a fasta file, with key = accession(name) and 
    value = sequence
    params
        fasta_file: file, str
    returns
        fasta_dict: dict
    """
    fasta_dict = {}
    for ref_name, _, seq in read_fasta(fasta_file):
        fasta_dict[ref_name] = seq
    return fasta_dict

In [7]:
fasta_file = 'GCF_000002985.6_WBcel235_rna.fna.gz'
contigs_file = "contigs.fa"

# build two dicts
ref_dict = fasta_to_dict(fasta_file)
ass_dict = fasta_to_dict(contigs_file)

has_error = False
occurences_file = "occurences.bed"
for ref_name, start, end, ass_name in read_bed(occurences_file):
    if not ref_dict[ref_name][start: end] == ass_dict[ass_name]:
        has_error = True
        print("Error")
        break
if not has_error:
    print("info in {} checked.".format(occurences_file))

info in occurences.bed checked.


An example of the first 10 lines in the "occurences.bed"

In [8]:
count = 0
for result in read_bed(occurences_file):
    print(result)
    count += 1
    if count > 10:
        break

('NM_001025782.2', 112, 212, 'IJKBZLZIXT')
('NM_001025782.2', 261, 378, 'CRTPPWYDNP')
('NM_001025782.2', 403, 559, 'BYKUOYBUAJ')
('NM_001025782.2', 565, 742, 'EBCKPZICNQ')
('NM_001025782.2', 773, 1037, 'MTQTWPVWVC')
('NM_001025782.2', 1040, 1140, 'HBWPKRVOET')
('NM_001025782.2', 1179, 1347, 'XFRFQVMBDT')
('NM_001025782.2', 1354, 1454, 'LMNPEMVYPB')
('NM_001025782.2', 1525, 1625, 'VRRUNQSPBG')
('NM_001025782.2', 1617, 1756, 'YXVQZWMSEW')
('NM_001025782.2', 1804, 1904, 'XAXXMZABCM')


### b) Peut-il exister des séquences qui ne peuvent pas être assemblée correctement dans la référence (avec k = 21)? Le cas échéant, en fournir un exemple.

**Answer b)**

we created a list "failures" to record all no-matched assembled sequences. We can check if this list has contents.

In [9]:
failures = []
with open("failure.bed", "rt") as f:
    for line in f:
        identifier, assem_seq = line.rstrip().split('\t')
        failures.append((identifier, assem_seq))
            
if len(failures) == 0:
    print("all assembled sequences are correctly assembled")
    print("each of them matches a or part of a sequence in FASTA")
else:
    print("first 2 sequences that didn't match any references are listed here:")
    for identifier, assem_seq in failures[:2]:
        print(identifier, " ", assem_seq, len(assem_seq))

first 2 sequences that didn't match any references are listed here:
AAAJMNNKVK   TATTATTACGAAGAAAAATACTCAAGCTTCTCACGGGACCTGATAGATCATCCGATGCGTTACATTCCCAAAATAGA 77
AADUMCEOYZ   GGAGCCGCCGGAGAAGACGGAACTCCAGGAGAGCCAGGAACTGACGGAGAGGATGGAGGAAACGGTATGCCTGGAGTCGCCCAGATGGCTATGGATGCTAATGGAGGAGCTTGCATCAAGTGCCCAGCCGGAGAGCCAGGACCACCAGGACCAGATGGACAGCCTGGAGCTCCAGGACCAGACGGACAGCCAGGAGCCGGTGGAACTACCAGCACCAACCAACCTGGACCTCCAGGACCAGCAGGACCACCTGGGCCAGCCGGACCAGCAGGAGAGGACGCGTATGCTCAACCATCACCAGCCGGAACACCAGGACCACCAGGACCAGAAGGACCAGCGGGACCAGCAGGACCAGCAGGAAACGCTGGAGCTGATGGAGAGGCCGGAGCTGAAGGACCAGCTGGAGAGGCTGGACCTCCAGGACCCCCAGGACCAGACGGAAAGCCTGGAGCTGCTGGATCTGACGGAGAAGCCGGAGCCCCTGGAACAAAGACCACAAATGCACCAGGAGAACCAGGACCAGCCGGACCACCAGGACCAGCCGGACCAGCTGGACCAAAGGGACCAAACGGAAACCCAGGATCCGACGGACAGCCAGGAGCCCCAGGACCAGCTGGACCAGAAGGAGATGCTGGAGCACCAGGAGACGCCGGAGCACCAGGACAGCCAGGGCATCCCGGATCCCCTGGAAACCCAGGACGTGGAGGGCAACGCAGCCGTGGACTTCCAGGACCATCAGGAAGACCGGGACCACAAGGACCGGCTGGAGGACCAGGACAGCCAGGACAGGATGGAGGATCTGGAGGATCGGGAGCT 816


### c) Bonus:  Proposer une structure de données appropriée pour effectuer une comparaison efficace des séquences assemblées avec celles de référence. La complexité doit être strictement inférieure à la méthode proposée en 4a.

**Answer c)**

Method `str.find()` built-in python combines the advantages of two algorithm (Boyer Moore, and Horspool) and uses bloom filter algorithm; all these techniques make the str.find() method very efficase and fast. We tried "sunday" algorithm, and another search algorithm based on python built-in hash function. Unforturnately, they are both less efficase than `str.find()` method. The implementation sunday algorithm and hash method is shown below.

As `str.find()` method is a general method for all typs of strings. We come to an idea if we can use one of the characteristic a DNA sequence: only four different charcters may occur in a string, to design a faster search algorithm. But we haven't found a good way to impliment it yet.

In [10]:
# sunday search algorithm
def sunday_occ(P):
    """get position index of a char in pattern string
    params
        P: pattern string
    returns
        occ: list of index
    """
    occ = {} # AGCT
    for j in range(len(P)):
        occ[P[j]] = j
    return occ

def sunday_find(T, P, occ):
    m, n = len(T), len(P)
    if n == 0 or m < n: return -1
    #occ = sunday_occ(P)
    i = 0
    while i <= m - n:
        #print("\nround", i)
        #print(T)
        #print(" "*i+P)
        if P == T[i: i+n]:
            return i
        i += n
        #print(" "*i + P)
        if i < m:
            i -= occ.get(T[i], 0)
            #print(" "*i + P)
    return -1

In [11]:
# hash and compare
def hash_find(T, P):
    m, n = len(T), len(P)
    if n == 0 or m < n: return -1
    hash_p = hash(P)
    for i in range(m-n+1):
        s = T[i: i+n]
        hash_s = hash(s)
        if hash_p == hash_s and P == s:
            return i
    return -1   

Test the time needed for each algorithm.

Generate some Texts(`Ts`) and a pattern(`P`), then find the pattern in each text.

In [12]:
import random
Ts = []
for i in range(10000):
    Ts.append(''.join(random.choices("ACGT", k = 3000)))
P = ''.join(random.choices("ACGT", k = 300))

In [13]:
%%time
for T in Ts:
    T.find(P)

CPU times: user 201 ms, sys: 16 µs, total: 201 ms
Wall time: 199 ms


In [14]:
%%time
occ = sunday_occ(P)
for T in Ts:
    sunday_find(T, P, occ)

CPU times: user 1.82 s, sys: 25 µs, total: 1.82 s
Wall time: 1.82 s


In [15]:
%%time
for T in Ts:
    hash_find(T, P)

CPU times: user 10.6 s, sys: 3.96 ms, total: 10.6 s
Wall time: 10.6 s


Among three search methods, python built-in search algorithm is the fastest. By the time we submitted this report, we havn't found a faster search algorithm yet.